In [190]:
import pandas as pd
import numpy as np
import datetime
import category_encoders as ce
from category_encoders.one_hot import OneHotEncoder

In [191]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [192]:
#!pip install category_encoders

In [193]:
df = pd.read_csv('../../dados/dadosCompletos_2021.csv', sep=';')

In [194]:
pd.set_option('display.max_columns',999)
pd.set_option("display.max_rows", 999)

In [195]:
df.sample(5)

,id_animal,estabelecimento_municipio,data_abate,estabelecimento_identificador,identificador_lote,tipificacao,maturidade,acabamento,peso,motivo_desclassificacao,classificacao_estabelecimento,outros_incentivos,pratica_recuperacao_pastagem,fertiirrigacao,ilp,ilpf,ifp,fabrica_racao,identificacao_individual,regua_de_manejo,bpa,participa_aliancas_mercadologicas,qual_alianca,boa_cobertura_vegetal,erosao_laminar_ou_sulco_20,rastreamento_sisbov,lista_trace,data12m,data6m,data3m,data1m,data7d,tot7d_chuva,med7d_tempinst,med7d_tempmax,med7d_tempmin,med7d_umidinst,med7d_umidmax,med7d_umidmin,med7d_velventomax,med7d_formituinst,med7d_formitumax,med7d_num_pixels,med7d_ndvi,med7d_evi,med7d_prer_soja,med7d_prer_milho,med7d_prer_boi,tot1m_chuva,med1m_tempinst,med1m_tempmax,med1m_tempmin,med1m_umidinst,med1m_umidmax,med1m_umidmin,med1m_velventomax,med1m_formituinst,med1m_formitumax,med1m_num_pixels,med1m_ndvi,med1m_evi,med1m_prer_soja,med1m_prer_milho,med1m_prer_boi,tot3m_chuva,med3m_tempinst,med3m_tempmax,med3m_tempmin,med3m_umidinst,med3m_umidmax,med3m_umidmin,med3m_velventomax,med3m_formituinst,med3m_formitumax,med3m_num_pixels,med3m_ndvi,med3m_evi,med3m_prer_soja,med3m_prer_milho,med3m_prer_boi,tot6m_chuva,med6m_tempinst,med6m_tempmax,med6m_tempmin,med6m_umidinst,med6m_umidmax,med6m_umidmin,med6m_velventomax,med6m_formituinst,med6m_formitumax,med6m_num_pixels,med6m_ndvi,med6m_evi,med6m_prer_soja,med6m_prer_milho,med6m_prer_boi,tot12m_chuva,med12m_tempinst,med12m_tempmax,med12m_tempmin,med12m_umidinst,med12m_umidmax,med12m_umidmin,med12m_velventomax,med12m_formituinst,med12m_formitumax,med12m_num_pixels,med12m_ndvi,med12m_evi,med12m_prer_soja,med12m_prer_milho,med12m_prer_boi,cnt7d_cl_itumax,cnt7d_cl_ituinst,cnt1m_cl_itumax,cnt1m_cl_ituinst,cnt3m_cl_itumax,cnt3m_cl_ituinst,cnt6m_cl_itumax,cnt6m_cl_ituinst,cnt12m_cl_itumax,cnt12m_cl_ituinst,ano,categoria,classificacao
984858,984859,IVINHEMA,29NOV2018,5929,76465,macho castrado,Dois dentes,gordura mediana - acima de 3 a ata 6 mm de esp...,285.5,CLASSIFICADO,21.0,NÃO,SIM,NÃO,SIM,NÃO,NÃO,NÃO,SIM,SIM,NÃO,NÃO,NaN,SIM,NÃO,NÃO,NaN,28NOV2017,30MAY2018,28AUG2018,29OCT2018,21NOV2018,26.600,25.47,31.10,20.81,69.19,87.50,46.88,16.70,74.32,85.79,6103.0,0.63,0.43,82.80,37.44,147.04,101.03,24.72,30.97,19.87,69.37,87.39,45.39,18.02,73.28,85.54,6103.0,0.64,0.44,84.53,36.17,146.18,606.61,23.84,30.07,19.26,70.24,87.57,46.76,17.95,71.93,83.97,6103.0,0.60,0.40,90.31,37.91,148.04,762.61,22.80,29.26,17.90,66.03,84.50,42.35,17.11,70.01,82.15,6103.0,0.55,0.35,88.90,38.78,144.79,1822.83,23.53,29.78,18.99,69.05,86.60,45.01,16.66,71.40,83.41,6103.0,0.60,0.40,83.78,37.99,144.51,0.0,0.500,0.0645,0.4839,0.0968,0.6237,0.1475,0.7596,0.1129,0.6474,2018,AA,1
899868,899869,SANTA RITA DO PARDO,01NOV2018,5459,72500,femea,Dois dentes,gordura mediana - acima de 3 a ata 6 mm de esp...,208.5,CLASSIFICADO,21.0,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,SIM,SIM,SIM,NÃO,SIM,ANGUS,SIM,NÃO,SIM,SIM,31OCT2017,02MAY2018,31JUL2018,01OCT2018,24OCT2018,110.950,24.19,29.75,20.45,73.66,94.25,52.25,20.97,72.82,84.61,41689.0,0.62,0.41,87.04,34.58,147.18,274.45,24.21,30.30,19.85,71.27,92.84,46.71,21.19,72.73,85.35,41689.0,0.61,0.39,90.53,36.43,148.56,568.42,23.41,29.84,18.59,65.81,89.66,41.49,21.38,70.91,83.90,41689.0,0.49,0.28,91.74,39.29,147.45,638.89,23.08,29.72,18.00,62.82,87.05,38.33,19.72,70.14,83.26,41689.0,0.47,0.27,89.11,39.70,143.94,1398.30,24.06,30.38,19.39,66.66,89.98,41.01,19.21,72.00,84.96,41689.0,0.55,0.34,83.02,37.66,144.11,0.0,0.750,0.0645,0.5806,0.0860,0.7312,0.0929,0.7923,0.0628,0.6120,2018,AA,1
73673,73674,RIBAS DO RIO PARDO,02AUG2017,4391,15075,macho inteiro,Quatro dentes,gordura mediana - acima de 3 a ata 6 mm de esp...,307.5,DESCLASSIFICADO PELA MATURIDADE,30.0,NÃO,NÃO,NÃO,NÃO,NÃO,NÃO,SIM,SIM,SIM,NÃO,SIM,ASPNP,SIM,NÃO,SIM,SIM,01AUG2016,31JAN2017,01MAY2017,02JUL2017,25JUL2017,0.000,21.77,31.96,11.71,55.19,93.00,22.50,14.22,67.92,88.29,106541.0,0.38,0.22,71.79,25.50,124.67,26.48,21.05,31.04,12.36,61.16,93.07,26.69,13.62,67.33,86.75,106541.0,0.43,0.2

In [196]:
categorical_attributes = list(df.select_dtypes(include=['object']).columns)

In [197]:
categorical_attributes

['estabelecimento_municipio',
 'data_abate',
 'tipificacao',
 'maturidade',
 'acabamento',
 'motivo_desclassificacao',
 'outros_incentivos',
 'pratica_recuperacao_pastagem',
 'fertiirrigacao',
 'ilp',
 'ilpf',
 'ifp',
 'fabrica_racao',
 'identificacao_individual',
 'regua_de_manejo',
 'bpa',
 'participa_aliancas_mercadologicas',
 'qual_alianca',
 'boa_cobertura_vegetal',
 'erosao_laminar_ou_sulco_20',
 'rastreamento_sisbov',
 'lista_trace',
 'data12m',
 'data6m',
 'data3m',
 'data1m',
 'data7d',
 'categoria']

In [198]:
#numerical_attributes = list(data_news.select_dtypes(include=['float64', 'int64']).columns)

### 1. Estabelecimento Município

In [199]:
csv_municipios = pd.read_csv('../../dados/municipios.csv', encoding='utf-8', delimiter=',')

In [200]:
csv_municipios.head()

,municipios,longitude,latitude
0,RIBAS DO RIO PARDO,"-53,761678","-20,443676"
1,CAMAPUA,"-54,043309","-19,53526"
2,BRASILANDIA,"-52,036297","-21,247118"
3,NOVA ANDRADINA,"-53,351574","-22,248759"
4,SANTA RITA DO PARDO,"-52,828071","-21,305377"


In [201]:
df = pd.merge(df, csv_municipios, how='left', left_on='estabelecimento_municipio', right_on='municipios')

In [202]:
df.drop(['estabelecimento_municipio', 'municipios'], axis=1, inplace=True)
#df.index.name = 'index'

In [203]:
selecao = df[df['longitude'].isnull()]

In [205]:
selecao.head()

,id_animal,data_abate,estabelecimento_identificador,identificador_lote,tipificacao,maturidade,acabamento,peso,motivo_desclassificacao,classificacao_estabelecimento,outros_incentivos,pratica_recuperacao_pastagem,fertiirrigacao,ilp,ilpf,ifp,fabrica_racao,identificacao_individual,regua_de_manejo,bpa,participa_aliancas_mercadologicas,qual_alianca,boa_cobertura_vegetal,erosao_laminar_ou_sulco_20,rastreamento_sisbov,lista_trace,data12m,data6m,data3m,data1m,data7d,tot7d_chuva,med7d_tempinst,med7d_tempmax,med7d_tempmin,med7d_umidinst,med7d_umidmax,med7d_umidmin,med7d_velventomax,med7d_formituinst,med7d_formitumax,med7d_num_pixels,med7d_ndvi,med7d_evi,med7d_prer_soja,med7d_prer_milho,med7d_prer_boi,tot1m_chuva,med1m_tempinst,med1m_tempmax,med1m_tempmin,med1m_umidinst,med1m_umidmax,med1m_umidmin,med1m_velventomax,med1m_formituinst,med1m_formitumax,med1m_num_pixels,med1m_ndvi,med1m_evi,med1m_prer_soja,med1m_prer_milho,med1m_prer_boi,tot3m_chuva,med3m_tempinst,med3m_tempmax,med3m_tempmin,med3m_umidinst,med3m_umidmax,med3m_umidmin,med3m_velventomax,med3m_formituinst,med3m_formitumax,med3m_num_pixels,med3m_ndvi,med3m_evi,med3m_prer_soja,med3m_prer_milho,med3m_prer_boi,tot6m_chuva,med6m_tempinst,med6m_tempmax,med6m_tempmin,med6m_umidinst,med6m_umidmax,med6m_umidmin,med6m_velventomax,med6m_formituinst,med6m_formitumax,med6m_num_pixels,med6m_ndvi,med6m_evi,med6m_prer_soja,med6m_prer_milho,med6m_prer_boi,tot12m_chuva,med12m_tempinst,med12m_tempmax,med12m_tempmin,med12m_umidinst,med12m_umidmax,med12m_umidmin,med12m_velventomax,med12m_formituinst,med12m_formitumax,med12m_num_pixels,med12m_ndvi,med12m_evi,med12m_prer_soja,med12m_prer_milho,med12m_prer_boi,cnt7d_cl_itumax,cnt7d_cl_ituinst,cnt1m_cl_itumax,cnt1m_cl_ituinst,cnt3m_cl_itumax,cnt3m_cl_ituinst,cnt6m_cl_itumax,cnt6m_cl_ituinst,cnt12m_cl_itumax,cnt12m_cl_ituinst,ano,categoria,classificacao,longitude,latitude


### 2. Datas

In [206]:
df['data_abate'] = pd.to_datetime(df['data_abate'])

In [207]:
df['mes'] = df['data_abate'].dt.month 
df['dia'] = df['data_abate'].dt.day 

In [208]:
df['data12m'] = pd.to_datetime(df['data12m'])

In [209]:
df.drop(['qual_alianca', 'data6m','data3m','data1m','data7d'], axis=1, inplace=True)

### 3. Tipificação

In [210]:
df.tipificacao.value_counts()

macho inteiro     528127
femea             324069
macho castrado    255493
Name: tipificacao, dtype: int64

In [211]:
df['tipificacao'].astype('category')

0           macho inteiro
1          macho castrado
2          macho castrado
3          macho castrado
4          macho castrado
                ...      
1107684     macho inteiro
1107685     macho inteiro
1107686     macho inteiro
1107687     macho inteiro
1107688     macho inteiro
Name: tipificacao, Length: 1107689, dtype: category
Categories (3, object): ['femea', 'macho castrado', 'macho inteiro']

In [212]:
one_hot_enc = OneHotEncoder(cols=['tipificacao'])

In [213]:
one_hot_enc.fit_transform(df)

,id_animal,data_abate,estabelecimento_identificador,identificador_lote,tipificacao_1,tipificacao_2,tipificacao_3,maturidade,acabamento,peso,motivo_desclassificacao,classificacao_estabelecimento,outros_incentivos,pratica_recuperacao_pastagem,fertiirrigacao,ilp,ilpf,ifp,fabrica_racao,identificacao_individual,regua_de_manejo,bpa,participa_aliancas_mercadologicas,boa_cobertura_vegetal,erosao_laminar_ou_sulco_20,rastreamento_sisbov,lista_trace,data12m,tot7d_chuva,med7d_tempinst,med7d_tempmax,med7d_tempmin,med7d_umidinst,med7d_umidmax,med7d_umidmin,med7d_velventomax,med7d_formituinst,med7d_formitumax,med7d_num_pixels,med7d_ndvi,med7d_evi,med7d_prer_soja,med7d_prer_milho,med7d_prer_boi,tot1m_chuva,med1m_tempinst,med1m_tempmax,med1m_tempmin,med1m_umidinst,med1m_umidmax,med1m_umidmin,med1m_velventomax,med1m_formituinst,med1m_formitumax,med1m_num_pixels,med1m_ndvi,med1m_evi,med1m_prer_soja,med1m_prer_milho,med1m_prer_boi,tot3m_chuva,med3m_tempinst,med3m_tempmax,med3m_tempmin,med3m_umidinst,med3m_umidmax,med3m_umidmin,med3m_velventomax,med3m_formituinst,med3m_formitumax,med3m_num_pixels,med3m_ndvi,med3m_evi,med3m_prer_soja,med3m_prer_milho,med3m_prer_boi,tot6m_chuva,med6m_tempinst,med6m_tempmax,med6m_tempmin,med6m_umidinst,med6m_umidmax,med6m_umidmin,med6m_velventomax,med6m_formituinst,med6m_formitumax,med6m_num_pixels,med6m_ndvi,med6m_evi,med6m_prer_soja,med6m_prer_milho,med6m_prer_boi,tot12m_chuva,med12m_tempinst,med12m_tempmax,med12m_tempmin,med12m_umidinst,med12m_umidmax,med12m_umidmin,med12m_velventomax,med12m_formituinst,med12m_formitumax,med12m_num_pixels,med12m_ndvi,med12m_evi,med12m_prer_soja,med12m_prer_milho,med12m_prer_boi,cnt7d_cl_itumax,cnt7d_cl_ituinst,cnt1m_cl_itumax,cnt1m_cl_ituinst,cnt3m_cl_itumax,cnt3m_cl_ituinst,cnt6m_cl_itumax,cnt6m_cl_ituinst,cnt12m_cl_itumax,cnt12m_cl_ituinst,ano,categoria,classificacao,longitude,latitude,mes,dia
0,1,2017-02-09,1034,2,1,0,0,Dois dentes,magra - gordura ausente,283.0,DESCLASSIFICADO PELO ACABAMENTO,30.0,NÃO,SIM,NÃO,SIM,NÃO,NÃO,SIM,SIM,SIM,NÃO,SIM,SIM,NÃO,NÃO,NaN,2016-02-09,4.666667,21.29,28.04,16.94,74.99,90.6,47.2,18.29,68.55,81.12,NaN,NaN,NaN,74.11,36.19,145.18,120.57,21.99,27.79,18.51,80.49,93.00,55.80,17.83,70.08,81.06,NaN,NaN,NaN,75.70,35.68,147.26,323.53,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.70,640.03,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.7,1221.96,22.71,28.05,19.47,81.06,93.11,57.39,16.79,71.25,81.57,NaN,NaN,NaN,75.62,35.98,147.70,0.0,1.0,0.0,1.0000,0.0323,1.0000,0.0323,1.0000,0.0323,1.0000,2017,D,0,"-53,01289439","-19,02351863",2,9
1,2,2017-02-09,1034,2,0,1,0,Quatro dentes,gordura mediana - acima de 3 a ata 6 mm de esp...,268.8,CLASSIFICADO,30.0,NÃO,SIM,NÃO,SIM,NÃO,NÃO,SIM,SIM,SIM,NÃO,SIM,SIM,NÃO,NÃO,NaN,2016-02-09,4.666667,21.29,28.04,16.94,74.99,90.6,47.2,18.29,68.55,81.12,NaN,NaN,NaN,74.11,36.19,145.18,120.57,21.99,27.79,18.51,80.49,93.00,55.80,17.83,70.08,81.06,NaN,NaN,NaN,75.70,35.68,147.26,323.53,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.70,640.03,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.7,1221.96,22.71,28.05,19.47,81.06,93.11,57.39,16.79,71.25,81.57,NaN,NaN,NaN,75.62,35.98,147.70,0.0,1.0,0.0,1.0000,0.0323,1.0000,0.0323,1.0000,0.0323,1.0000,2017,BBB,1,"-53,01289439","-19,02351863",2,9
2,3,2017-02-09,1034,2,0,1,0,Dois dentes,gordura mediana - acima de 3 a ata 6 mm de esp...,282.4,CLASSIFICADO,30.0,NÃO,SIM,NÃO,SIM,NÃO,NÃO,SIM,SIM,SIM,NÃO,SIM,SIM,NÃO,NÃO,NaN,2016-02-09,4.666667,21.29,28.04,16.94,74.99,90.6,47.2,18.29,68.55,81.12,NaN,NaN,NaN,74.11,36.19,145.18,120.57,21.99,27.79,18.51,80.49,93.00,55.80,17.83,70.08,81.06,NaN,NaN,NaN,75.70,35.68,147.26,323.53,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.70,640.03,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.7,1221.96,22.71,28.05,19.47,81.06,93.11,57.39,16.79,71.25,81.57,NaN,NaN,NaN,75.62

In [214]:
#df = pd.get_dummies(df, columns=['tipificacao'], prefix=['tipificacao'])

### 4. Maturidade

In [215]:
df.maturidade.value_counts()

Dois dentes       448579
Dente de leite    374686
Quatro dentes     204907
Seis dentes        50894
Oito dentes        28623
Name: maturidade, dtype: int64

In [216]:
df['maturidade'].replace(['Dois dentes'], 2, inplace=True)
df['maturidade'].replace(['Dente de leite'], 1, inplace=True)
df['maturidade'].replace(['Quatro dentes'], 4, inplace=True)
df['maturidade'].replace(['Seis dentes'], 6, inplace=True)
df['maturidade'].replace(['Oito dentes'], 8, inplace=True)

### 5. Acabamento

In [217]:
df['acabamento'].value_counts()

gordura mediana - acima de 3 a ata 6 mm de espessura      590250
gordura escassa - 1 a 3 mm de espessura                   443365
gordura uniforme - acima de 6 e ata 10 mm de espessura     68315
magra - gordura ausente                                     5490
gordura excessiva - acima de 10 mm de espessura              269
Name: acabamento, dtype: int64

In [218]:
df['acabamento'].replace(['magra - gordura ausente'], 1, inplace=True)
df['acabamento'].replace(['gordura escassa - 1 a 3 mm de espessura'], 2, inplace=True)
df['acabamento'].replace(['gordura mediana - acima de 3 a ata 6 mm de espessura'], 3, inplace=True)
df['acabamento'].replace(['gordura uniforme - acima de 6 e ata 10 mm de espessura'], 4, inplace=True)
df['acabamento'].replace(['gordura excessiva - acima de 10 mm de espessura'], 5, inplace=True)

### Substituindo os valores "Não"  e "Sim" por 0 e 1.

In [219]:
df = df.applymap(lambda x: 0 if "Não" in str(x) else x)
df = df.applymap(lambda x: 0 if "NÃO" in str(x) else x)

In [220]:
df = df.applymap(lambda x: 1 if "Sim" in str(x) else x)
df = df.applymap(lambda x: 1 if "SIM" in str(x) else x)

In [221]:
selecao = pd.DataFrame(data = df, columns = ['possui_outros_ incentivo','pratica_recuperacao_pastagem','fertiirrigacao', 'ilp', 'ilpf','ifp','fabrica_racao','identificacao_individual','regua_de_manejo','bpa','participa_de_aliancas_mercadolog','boa_cobertura_vegetal_com_baixa','erosao_laminar_ou_em_sulco_igua','rastreamento_sisbov','lista_trace'])

### 6. Motivo de desclassificação

In [222]:
df.motivo_desclassificacao.value_counts()

CLASSIFICADO                       924014
DESCLASSIFICADO PELA MATURIDADE    146786
DESCLASSIFICADO PELO PESO           30496
DESCLASSIFICADO PELO ACABAMENTO      4336
DESCLASSIFICADO PELO SIF             2057
Name: motivo_desclassificacao, dtype: int64

In [223]:
df['motivo_desclassificacao'].astype('category')

0          DESCLASSIFICADO PELO ACABAMENTO
1                             CLASSIFICADO
2                             CLASSIFICADO
3          DESCLASSIFICADO PELA MATURIDADE
4                             CLASSIFICADO
                        ...               
1107684    DESCLASSIFICADO PELA MATURIDADE
1107685    DESCLASSIFICADO PELA MATURIDADE
1107686    DESCLASSIFICADO PELA MATURIDADE
1107687    DESCLASSIFICADO PELA MATURIDADE
1107688    DESCLASSIFICADO PELA MATURIDADE
Name: motivo_desclassificacao, Length: 1107689, dtype: category
Categories (5, object): ['CLASSIFICADO', 'DESCLASSIFICADO PELA MATURIDADE', 'DESCLASSIFICADO PELO ACABAMENTO', 'DESCLASSIFICADO PELO PESO', 'DESCLASSIFICADO PELO SIF']

In [224]:
one_hot_enc = OneHotEncoder(cols=['tipificacao'])

In [225]:
one_hot_enc.fit_transform(df)

,id_animal,data_abate,estabelecimento_identificador,identificador_lote,tipificacao_1,tipificacao_2,tipificacao_3,maturidade,acabamento,peso,motivo_desclassificacao,classificacao_estabelecimento,outros_incentivos,pratica_recuperacao_pastagem,fertiirrigacao,ilp,ilpf,ifp,fabrica_racao,identificacao_individual,regua_de_manejo,bpa,participa_aliancas_mercadologicas,boa_cobertura_vegetal,erosao_laminar_ou_sulco_20,rastreamento_sisbov,lista_trace,data12m,tot7d_chuva,med7d_tempinst,med7d_tempmax,med7d_tempmin,med7d_umidinst,med7d_umidmax,med7d_umidmin,med7d_velventomax,med7d_formituinst,med7d_formitumax,med7d_num_pixels,med7d_ndvi,med7d_evi,med7d_prer_soja,med7d_prer_milho,med7d_prer_boi,tot1m_chuva,med1m_tempinst,med1m_tempmax,med1m_tempmin,med1m_umidinst,med1m_umidmax,med1m_umidmin,med1m_velventomax,med1m_formituinst,med1m_formitumax,med1m_num_pixels,med1m_ndvi,med1m_evi,med1m_prer_soja,med1m_prer_milho,med1m_prer_boi,tot3m_chuva,med3m_tempinst,med3m_tempmax,med3m_tempmin,med3m_umidinst,med3m_umidmax,med3m_umidmin,med3m_velventomax,med3m_formituinst,med3m_formitumax,med3m_num_pixels,med3m_ndvi,med3m_evi,med3m_prer_soja,med3m_prer_milho,med3m_prer_boi,tot6m_chuva,med6m_tempinst,med6m_tempmax,med6m_tempmin,med6m_umidinst,med6m_umidmax,med6m_umidmin,med6m_velventomax,med6m_formituinst,med6m_formitumax,med6m_num_pixels,med6m_ndvi,med6m_evi,med6m_prer_soja,med6m_prer_milho,med6m_prer_boi,tot12m_chuva,med12m_tempinst,med12m_tempmax,med12m_tempmin,med12m_umidinst,med12m_umidmax,med12m_umidmin,med12m_velventomax,med12m_formituinst,med12m_formitumax,med12m_num_pixels,med12m_ndvi,med12m_evi,med12m_prer_soja,med12m_prer_milho,med12m_prer_boi,cnt7d_cl_itumax,cnt7d_cl_ituinst,cnt1m_cl_itumax,cnt1m_cl_ituinst,cnt3m_cl_itumax,cnt3m_cl_ituinst,cnt6m_cl_itumax,cnt6m_cl_ituinst,cnt12m_cl_itumax,cnt12m_cl_ituinst,ano,categoria,classificacao,longitude,latitude,mes,dia
0,1,2017-02-09,1034,2,1,0,0,2,1,283.0,DESCLASSIFICADO PELO ACABAMENTO,30.0,0.0,1.0,0,1,0,0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,2016-02-09,4.666667,21.29,28.04,16.94,74.99,90.6,47.2,18.29,68.55,81.12,NaN,NaN,NaN,74.11,36.19,145.18,120.57,21.99,27.79,18.51,80.49,93.00,55.80,17.83,70.08,81.06,NaN,NaN,NaN,75.70,35.68,147.26,323.53,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.70,640.03,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.7,1221.96,22.71,28.05,19.47,81.06,93.11,57.39,16.79,71.25,81.57,NaN,NaN,NaN,75.62,35.98,147.70,0.0,1.0,0.0,1.0000,0.0323,1.0000,0.0323,1.0000,0.0323,1.0000,2017,D,0,"-53,01289439","-19,02351863",2,9
1,2,2017-02-09,1034,2,0,1,0,4,3,268.8,CLASSIFICADO,30.0,0.0,1.0,0,1,0,0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,2016-02-09,4.666667,21.29,28.04,16.94,74.99,90.6,47.2,18.29,68.55,81.12,NaN,NaN,NaN,74.11,36.19,145.18,120.57,21.99,27.79,18.51,80.49,93.00,55.80,17.83,70.08,81.06,NaN,NaN,NaN,75.70,35.68,147.26,323.53,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.70,640.03,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.7,1221.96,22.71,28.05,19.47,81.06,93.11,57.39,16.79,71.25,81.57,NaN,NaN,NaN,75.62,35.98,147.70,0.0,1.0,0.0,1.0000,0.0323,1.0000,0.0323,1.0000,0.0323,1.0000,2017,BBB,1,"-53,01289439","-19,02351863",2,9
2,3,2017-02-09,1034,2,0,1,0,2,3,282.4,CLASSIFICADO,30.0,0.0,1.0,0,1,0,0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,2016-02-09,4.666667,21.29,28.04,16.94,74.99,90.6,47.2,18.29,68.55,81.12,NaN,NaN,NaN,74.11,36.19,145.18,120.57,21.99,27.79,18.51,80.49,93.00,55.80,17.83,70.08,81.06,NaN,NaN,NaN,75.70,35.68,147.26,323.53,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.70,640.03,21.91,27.55,18.46,79.88,92.35,55.23,17.59,69.90,80.56,NaN,NaN,NaN,75.62,35.98,147.7,1221.96,22.71,28.05,19.47,81.06,93.11,57.39,16.79,71.25,81.57,NaN,NaN,NaN,75.62,35.98,147.70,0.0,1.0,0.0,1.0000,0.0323,1.0000,0.0323,1.0000,0.0323,1.0000,2017,AA,1,"-53,01289439","-19,02351863",2,9
3,4,2017-02-09,1034,2,0,1,0,6,3,271.4,DESCLASSIFICADO P

### 7. Fertiirrigação, ilp, ilpf e ifp

In [226]:
df.fertiirrigacao.value_counts()

0       959158
1        98695
NULO     49836
Name: fertiirrigacao, dtype: int64

In [227]:
df.ilp.replace('NULO', np.nan, inplace= True)

In [228]:
df.ilpf.replace('NULO', np.nan, inplace= True)

In [229]:
df.ifp.replace('NULO', np.nan, inplace= True)

### 8. Categoria

In [230]:
df.categoria.value_counts()

AA     381558
AAA    231793
BB     182830
D      182108
BBB     85432
C       43968
Name: categoria, dtype: int64

In [231]:
df['categoria'].replace(['AAA'], 1, inplace=True)
df['categoria'].replace(['AA'], 2, inplace=True)
df['categoria'].replace(['BBB'], 3, inplace=True)
df['categoria'].replace(['BB'], 4, inplace=True)
df['categoria'].replace(['C'], 5, inplace=True)
df['categoria'].replace(['D'], 6, inplace=True)